## Table of contents

1. **LLM Configuration**
   - Import ChatTogether from langchain_together
   - Set up the LLM using ChatTogether with Meta-Llama-3.1-8B-Instruct-Turbo model

2. **Basic LLM Interaction**
   - Create a simple conversation with system and human messages
   - Invoke the LLM and display the response

3. **Router Chain Setup**
   - Import necessary components from langchain
   - Define templates for different types of inquiries:
     - Flight status
     - Baggage inquiry
     - Ticket booking
     - General inquiry

4. **Destination Chains Configuration**
   - Set up destination chains for each inquiry type
   - Create a fallback chain for unmatched queries

5. **Router Template Definition**
   - Define the MULTI_PROMPT_ROUTER_TEMPLATE
   - Create the router prompt and chain

6. **Multi-Prompt Chain Creation**
   - Combine router chain, destination chains, and default chain

7. **Testing the Chatbot**
   - Run sample queries to test different aspects of the chatbot:
     - Flight status inquiry
     - General airline safety inquiry
     - Ticket upgrade request


In [1]:
import warnings
warnings.filterwarnings("ignore")

## Basic LLM Interaction

In [2]:
## setting up the language model
import os
from dotenv import load_dotenv
from langchain_together import ChatTogether

## load the environment variables
load_dotenv(override=True)

llm = ChatTogether(api_key=os.getenv("TOGETHERAI_API_KEY"),temperature=0.0, model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo")

messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
ai_msg

AIMessage(content="Je m'adore programmer.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 56, 'total_tokens': 64}, 'model_name': 'meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo', 'system_fingerprint': None, 'finish_reason': 'eos', 'logprobs': None}, id='run-6d495c7c-55e1-4013-b358-c073dfc9c158-0', usage_metadata={'input_tokens': 56, 'output_tokens': 8, 'total_tokens': 64})

In [3]:
ai_msg.content

"Je m'adore programmer."

### Chains in Langchain

In LangChain, a "chain" refers to a sequence of steps where each step involves processing some input and producing some output, typically using a large language model (LLM) or other tools. 

Chains allow for creating complex workflows by linking together multiple prompts or actions. This helps in building more sophisticated and structured applications, enhancing modularity, and reusability.

They further provide a structured way to manage interactions with LLMs and other tools, making it easier to handle complex logic and stateful interactions.

In [4]:
# from langchain.chains.router import MultiPromptChain  # Import for creating a chain with multiple prompts
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser  # Imports for routing logic in LLM chains
from langchain.prompts import PromptTemplate, ChatPromptTemplate  # Imports for creating prompt templates
from langchain import LLMChain

In [5]:

# Define templates for handling different types of user queries

flight_status_template = """You are a helpful airline customer service representative.
Answer the following query about flight status. Provide accurate, concise, and friendly information.
Here is the query:
{input}"""

baggage_inquiry_template = """You are a knowledgeable airline customer service representative.
Provide information regarding baggage policies, lost baggage, or baggage fees.
Here is the query:
{input}"""

ticket_booking_template = """You are an airline booking agent. Assist the customer in booking, modifying, or canceling a flight.
Here is the request:
{input}"""

general_inquiry_template = """You are a customer service representative at an airline.
Answer the following general inquiry about the airline, its services, or policies.
Here is the question:
{input}"""



In [6]:

# Define prompt information for different routes
prompt_infos = [
    {
        "name": "flight_status",
        "description": "Handles inquiries related to flight status.",
        "prompt_template": flight_status_template
    },
    {
        "name": "baggage_inquiry",
        "description": "Good for responding to questions related to baggage, including fees and lost items.",
        "prompt_template": baggage_inquiry_template
    },
    {
        "name": "ticket_booking",
        "description": "Handles ticket booking, modification, or cancellation requests.",
        "prompt_template": ticket_booking_template
    },
    {
        "name": "general_inquiry",
        "description": "Good for answering general inquiries about the airline or its services or handling feedbacks.",
        "prompt_template": general_inquiry_template
    }
]


## Create destination chain for each query prompt template

In [7]:
# Destination Chains Setup: A loop iterates over each item in prompt_infos,
# creates a chain for each feedback type,
# and stores it in the destination_chains dictionary.
# Each chain is created by combining the ChatPromptTemplate with LLMChain.


destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

# Create destinations string for router template
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)


/var/folders/56/z26yjgq567gdhj2y_0sl53yh0000gn/T/ipykernel_63435/2975751954.py:12: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


In [8]:
destination_chains

{'flight_status': LLMChain(prompt=ChatPromptTemplate(input_variables=['input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='You are a helpful airline customer service representative.\nAnswer the following query about flight status. Provide accurate, concise, and friendly information.\nHere is the query:\n{input}'))]), llm=ChatTogether(client=<openai.resources.chat.completions.Completions object at 0x10f492f40>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x10f4a6130>, model_name='meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo', temperature=0.0, openai_api_key=SecretStr('**********'), together_api_key=SecretStr('**********'))),
 'baggage_inquiry': LLMChain(prompt=ChatPromptTemplate(input_variables=['input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='You are a knowledgeable airline customer service representative.\nProvide information regarding baggage pol

### Router chains

In LangChain, Router Chains are used to dynamically select and route the input to different sub-chains or components based on the input's characteristics or conditions. This is useful when you need to handle different types of inputs or tasks with specialized processing logic.



In [9]:

# Create a fallback chain for any queries that don't fit a specific category
default_prompt = ChatPromptTemplate.from_template("reply to this {input} gracefully")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

# Define router template
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input, select the prompt best suited for the input.
You will be given the names of the available prompts and a description of what the prompt is best suited for.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ original input
}}}}
<< CANDIDATE PROMPTS >> {destinations}

<< INPUT >> {{input}}

<< OUTPUT (remember to include the ```json)>>"""

Router Template: This template instructs the model on how to choose the appropriate feedback chain based on the input. It describes the available prompt options and asks the model to select the most suitable one.

In [10]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)

router_prompt = PromptTemplate( template=router_template, input_variables=["input"], output_parser=RouterOutputParser(), )

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

## Combining the chains with `MultiPromptChain`


In [11]:
from langchain.chains.router import MultiPromptChain

chain = MultiPromptChain(router_chain=router_chain, destination_chains=destination_chains, default_chain=default_chain, verbose=True )

/var/folders/56/z26yjgq567gdhj2y_0sl53yh0000gn/T/ipykernel_63435/2647798521.py:3: LangChainDeprecationWarning: Use RunnableLambda to select from multiple prompt templates. See example in API reference: https://api.python.langchain.com/en/latest/chains/langchain.chains.router.multi_prompt.MultiPromptChain.html
  chain = MultiPromptChain(router_chain=router_chain, destination_chains=destination_chains, default_chain=default_chain, verbose=True )


## Testing it with different queries

In [12]:
chain.invoke( "What is the current status of flight AA123?")



> Entering new MultiPromptChain chain...
flight_status: {'input': 'What is the current status of flight AA123?'}
> Finished chain.


{'input': 'What is the current status of flight AA123?',
 'text': "I'd be happy to help you with the status of flight AA123. \n\nTo check the current status, I'll need to look up the flight information. Can you please tell me the departure and arrival cities for flight AA123, as well as the date of travel? This will help me provide you with the most accurate and up-to-date information."}

In [13]:
chain.invoke("Can you tell me about your airline's safety protocols?")



> Entering new MultiPromptChain chain...
general_inquiry: {'input': "Can you tell me about your airline's safety protocols?"}
> Finished chain.


{'input': "Can you tell me about your airline's safety protocols?",
 'text': "Thank you for choosing to fly with us. I'd be happy to provide you with information about our airline's safety protocols.\n\nAt [Airline Name], the safety of our passengers and crew is our top priority. We take every precaution to ensure that our flights are operated safely and efficiently. Here are some of the key safety protocols we have in place:\n\n1. **Regular Maintenance**: Our aircraft undergo regular maintenance checks to ensure that they are airworthy and meet the highest safety standards. Our maintenance team conducts thorough inspections and repairs to prevent any potential issues.\n\n2. **Pilot Training**: Our pilots undergo rigorous training and must meet the highest standards set by regulatory bodies. They are trained to handle emergency situations and are equipped with the latest technology to ensure safe flight operations.\n\n3. **Safety Briefings**: Before every flight, our crew provides a co

In [14]:
chain.invoke("how can i upgrade my flight ticket that is already booked.")



> Entering new MultiPromptChain chain...
ticket_booking: {'input': 'how can i upgrade my flight ticket that is already booked.'}
> Finished chain.


{'input': 'how can i upgrade my flight ticket that is already booked.',
 'text': "To upgrade your existing flight ticket, I'll need to check a few things first. Can you please provide me with the following information:\n\n1. Your booking reference number or ticket number?\n2. Your name as it appears on the ticket?\n3. The flight number and date of travel?\n4. The class of service you're currently booked in (economy, premium economy, business, or first class)?\n5. The class of service you'd like to upgrade to?\n\nAdditionally, I'll need to let you know that upgrades are subject to availability and may incur an additional fee. If there are any available upgrades, I can provide you with the details and prices.\n\nAlso, please note that some airlines have specific upgrade policies, such as:\n\n- Upgrades may be available at check-in or at the gate, depending on the airline's policy.\n- Some upgrades may require a higher fare or a fee to be paid.\n- Upgrades may not be available on all flig